In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
from glob import glob

# ⚙️ 데이터셋 yaml 정의
data_yaml = """\
path: /content/drive/MyDrive/Cleargrasp_Dataset
train: train/images
val: val/images

names:
  0: transparent_object
"""

yaml_path = "cleargrasp_seg_data.yaml"
with open(yaml_path, 'w') as f:
    f.write(data_yaml)

# ⚙️ 마스크를 YOLOv11 폴리곤 형식으로 변환
def convert_mask_to_txt(mask_path, txt_path, target_pixel_value=255, class_id=0):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape

    binary = (mask == target_pixel_value).astype(np.uint8) * 255
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    with open(txt_path, 'w') as f:
        for cnt in contours:
            if len(cnt) < 3:
                continue  # polygon은 최소 3개 점 필요
            normalized = [f"{pt[0][0] / w:.6f} {pt[0][1] / h:.6f}" for pt in cnt]
            line = f"{class_id} " + " ".join(normalized) + "\n"
            f.write(line)
    print(f"📄 {os.path.basename(txt_path)}: {len(contours)}개 contour 처리 완료")

# ⚙️ train, val 마스크에서 txt 라벨 생성
splits = ['train', 'val']
for split in splits:
    mask_dir = f"/content/drive/MyDrive/Cleargrasp_Dataset/{split}/segmentation-masks"
    label_dir = f"/content/drive/MyDrive/Cleargrasp_Dataset/{split}/labels"
    os.makedirs(label_dir, exist_ok=True)

    mask_paths = glob(f"{mask_dir}/*.png")
    print(f"🔍 {split} 세트에서 {len(mask_paths)}개 마스크 처리 시작...")

    for i, mask_path in enumerate(mask_paths):
        filename = os.path.splitext(os.path.basename(mask_path))[0]
        txt_path = os.path.join(label_dir, f"{filename}.txt")

        if os.path.exists(txt_path):
            print(f"⏩ [{i+1}/{len(mask_paths)}] {filename}.txt 이미 존재 - 건너뜀")
            continue

        convert_mask_to_txt(mask_path, txt_path)
        print(f"✅ [{i+1}/{len(mask_paths)}] {filename}.txt 생성 완료")

    print(f"🎉 {split} 세트 라벨 파일 생성 완료!\n")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔍 train 세트에서 0개 마스크 처리 시작...
🎉 train 세트 라벨 파일 생성 완료!

🔍 val 세트에서 705개 마스크 처리 시작...
⏩ [1/705] square-plastic-bottle-val (1).txt 이미 존재 - 건너뜀
⏩ [2/705] square-plastic-bottle-val (2).txt 이미 존재 - 건너뜀
⏩ [3/705] square-plastic-bottle-val (3).txt 이미 존재 - 건너뜀
⏩ [4/705] square-plastic-bottle-val (4).txt 이미 존재 - 건너뜀
⏩ [5/705] square-plastic-bottle-val (5).txt 이미 존재 - 건너뜀
⏩ [6/705] square-plastic-bottle-val (6).txt 이미 존재 - 건너뜀
⏩ [7/705] square-plastic-bottle-val (7).txt 이미 존재 - 건너뜀
⏩ [8/705] square-plastic-bottle-val (8).txt 이미 존재 - 건너뜀
⏩ [9/705] square-plastic-bottle-val (9).txt 이미 존재 - 건너뜀
⏩ [10/705] square-plastic-bottle-val (10).txt 이미 존재 - 건너뜀
⏩ [11/705] square-plastic-bottle-val (12).tx

In [4]:
from glob import glob
import os

split = 'train'
mask_dir = f"/content/drive/MyDrive/Cleargrasp_Dataset/{split}/labels"
mask_paths = glob(f"{mask_dir}/*.txt")
print(f"📂 디렉토리 경로: {mask_dir}")
print(f"🔍 마스크 파일 수: {len(mask_paths)}")

if len(mask_paths) > 0:
    print("📑 예시 파일:", os.path.basename(mask_paths[0]))
else:
    print("🚫 마스크 파일이 없음!")


📂 디렉토리 경로: /content/drive/MyDrive/Cleargrasp_Dataset/train/labels
🔍 마스크 파일 수: 45454
📑 예시 파일: stemless-plastic-champagne-glass-train (5381).txt


In [ ]:
'''
#데이터가 너무 많아 쓰지 않음
import os
from glob import glob
from tqdm import tqdm
import shutil

# 디렉토리 설정
src_img_root  = '/content/drive/MyDrive/Cleargrasp_Dataset/train/rgb-imgs'
src_label_root = '/content/drive/MyDrive/Cleargrasp_Dataset/train/labels'
dst_img_root  = '/content/yoloseg_dataset/images/train'
dst_label_root = '/content/yoloseg_dataset/labels/train'

src_img_root2  = '/content/drive/MyDrive/Cleargrasp_Dataset/val/rgb-imgs'
src_label_root2 = '/content/drive/MyDrive/Cleargrasp_Dataset/val/labels'
dst_img_root2  = '/content/yoloseg_dataset/images/val'
dst_label_root2 = '/content/yoloseg_dataset/labels/val'

# 디렉토리 생성
os.makedirs(dst_img_root, exist_ok=True)
os.makedirs(dst_label_root, exist_ok=True)
os.makedirs(dst_img_root2, exist_ok=True)
os.makedirs(dst_label_root2, exist_ok=True)

# ✅ 함수: 이미지 & 라벨 복사
def copy_images_and_labels(src_img_dir, src_label_dir, dst_img_dir, dst_label_dir):
    image_paths = sorted(glob(os.path.join(src_img_dir, '*.png')) + glob(os.path.join(src_img_dir, '*.jpg')))

    for img_path in tqdm(image_paths):
        img_name = os.path.basename(img_path)
        name_wo_ext = os.path.splitext(img_name)[0]

        # 이미지 복사
        shutil.copy(img_path, os.path.join(dst_img_dir, img_name))

        # 라벨 존재할 경우 복사
        label_path = os.path.join(src_label_dir, f"{name_wo_ext}.txt")
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dst_label_dir, f"{name_wo_ext}.txt"))

# 🧠 적용
copy_images_and_labels(src_img_root,  src_label_root,  dst_img_root,  dst_label_root)
copy_images_and_labels(src_img_root2, src_label_root2, dst_img_root2, dst_label_root2)
'''

In [5]:
# ⚙️ 모델 로드 및 학습
model = YOLO('yolo11n-seg.pt')  # pretrained model

model.train(
    data=yaml_path,
    epochs=2,
    imgsz=512,
    batch=32,
)


100%|██████████| 5.90M/5.90M [00:00<00:00, 344MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cleargrasp_seg_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

100%|██████████| 755k/755k [00:00<00:00, 120MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n-seg summary: 203 layers, 2,842,803 parameters, 2,842,787 gradients, 10.4 GFLOPs

Transferred 510/561 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/12 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.25it/s]


                   all        705       1607      0.888      0.901      0.951      0.858      0.901      0.914      0.959       0.81
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/segment/train


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a359c164750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [8]:
import os
import shutil
from google.colab import drive

# Google 드라이브 마운트
drive.mount('/content/drive')

# 원본 경로
source_path = '/content/runs'

# 목적지 경로
destination_path = '/content/drive/MyDrive/yolo_seg_result'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
from ultralytics import YOLO
from tqdm import tqdm
import cv2
import os
import numpy as np
import torch

def calculate_pixel_accuracy_yolo(
    model_path: str,
    test_image_dir: str,
    test_mask_dir: str,
    confidence_threshold: float = 0.5,
    mask_threshold: float = 0.5
) -> float:

    try:
        model = YOLO(model_path)
        if torch.cuda.is_available():
            model.to("cuda")
        else:
            print("💡 GPU를 사용할 수 없습니다. CPU 모드로 실행합니다.")
        model.eval()
    except Exception as e:
        print(f"❌ 모델 로드 중 오류 발생: {e}")
        return -1.0

    image_files = sorted([f for f in os.listdir(test_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    if not image_files:
        print(f"⚠️ {test_image_dir} 경로에 이미지가 없습니다.")
        return 0.0

    total_correct_pixels = 0
    total_pixels_evaluated = 0

    print(f"🚀 {len(image_files)}개의 테스트 이미지에 대해 픽셀 정확도를 계산합니다...")

    for img_file in tqdm(image_files, desc="테스트 진행 중"):
        img_path = os.path.join(test_image_dir, img_file)
        base_name = os.path.splitext(img_file)[0]
        mask_path = os.path.join(test_mask_dir, base_name + '.png')

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ 이미지 로드 실패: {img_path}. 건너뜝니다.")
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if gt_mask is None:
            print(f"⚠️ Ground Truth 마스크 로드 실패: {mask_path}. 해당 이미지에 대한 마스크가 없거나 손상되었습니다. 건너뜝니다.")
            continue

        gt_mask = (gt_mask == 255).astype(np.uint8)

        pred_mask = np.zeros_like(gt_mask, dtype=np.uint8)

        try:
            with torch.no_grad():
                results = model.predict(img_rgb, conf=confidence_threshold, verbose=False)

            if results and results[0].masks is not None and len(results[0].masks.data) > 0:
                raw_pred_mask = results[0].masks.data[0].cpu().numpy()
                pred_mask = (raw_pred_mask > mask_threshold).astype(np.uint8)

                if pred_mask.shape != gt_mask.shape:
                    pred_mask = cv2.resize(pred_mask, (gt_mask.shape[1], gt_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

        except Exception as e:
            print(f"⚠️ 예측 또는 마스크 처리 중 오류 발생 (파일: {img_file}): {e}. 해당 이미지를 건너뜝니다.")
            continue

        total_correct_pixels += np.sum(pred_mask == gt_mask)
        total_pixels_evaluated += gt_mask.size

    pixel_accuracy = (total_correct_pixels / total_pixels_evaluated) if total_pixels_evaluated > 0 else 0.0
    print(f"\n🎯 테스트 세트 픽셀 정확도: {pixel_accuracy * 100:.2f}%")
    return pixel_accuracy

# --- 실행 예시 ---
if __name__ == "__main__":
    MODEL_PATH = "/content/drive/MyDrive/yolo_seg_result/segment/train/weights/best.pt"
    TEST_IMAGE_DIR = "/content/drive/MyDrive/Cleargrasp_Dataset/test/images"
    TEST_MASK_DIR = "/content/drive/MyDrive/Cleargrasp_Dataset/test/segmentation-masks"

    accuracy = calculate_pixel_accuracy_yolo(
        model_path=MODEL_PATH,
        test_image_dir=TEST_IMAGE_DIR,
        test_mask_dir=TEST_MASK_DIR,
        confidence_threshold=0.25,
        mask_threshold=0.5
    )
    print(f"최종 계산된 픽셀 정확도: {accuracy * 100:.2f}%")

🚀 521개의 테스트 이미지에 대해 픽셀 정확도를 계산합니다...


테스트 진행 중: 100%|██████████| 521/521 [06:37<00:00,  1.31it/s]


🎯 테스트 세트 픽셀 정확도: 95.32%
최종 계산된 픽셀 정확도: 95.32%
